In [85]:
import os
from dotenv import load_dotenv
load_dotenv('key.env')  

key_string = os.getenv('open_ai_API_Key')

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import StructuredTool, Tool
from langchain_openai import ChatOpenAI

from langchain.tools import tool

In [87]:

from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI


In [88]:
class Extract_toolArgs(BaseModel):
    a: str = Field(description="Company name")
    b: str = Field(description="item")
    c: str = Field(description="Year_1")
    d: str = Field(description="Year_2")

In [89]:
class Extracted_toolArgs(BaseModel):
    year : list = Field(description="List of years")
    value : list = Field(discription = "List of Values")

In [90]:
import json
@tool(args_schema=Extract_toolArgs)
def Extract_Tool(a: str, b: str, c:str, d:str) -> Extract_toolArgs:
    """extracts specific data for a company from Year_1 to Year_2."""
    with open("data_soo.json","r") as json_file:
        data = json.load(json_file)
    
    a = a.upper()
    b = b.lower()
    json_data = next(item for item in data[a] if item["Item"] == b)
    
    year_data =[]
    cash_data = []
    
    # Ensure c and d represent years in the correct order
    if int(c) > int(d):
        c, d = d, c
    
    for year in range(int(c), int(d) + 1):
        value = json_data.get(str(year))
        if value is not None:
            cash_data.append(value)
            year_data.append(year)

            
    
    if len(cash_data) == 0:
        raise ValueError(f"Data for the years {c} to {d} is not available.")
    
    return cash_data,year_data




In [91]:
class GrowthToolArgs(BaseModel):
    sales_data : list = Field(description="list of sales data")
    year_data:list = Field(description="list of years of sales data")


In [92]:
@tool(args_schema=GrowthToolArgs)
def Growth_Tool(sales_data: list, year_data: list) -> list[float]:
    """
    Calculates year-on-year (YoY) growth for 'Net Sales' for a specific company.
    This function receives year data and sales data list.

    Arguments:
    - sales_data: List of dictionaries containing year and corresponding sales.
    - year_data: List of years corresponding to the sales data.
    
    Returns:
    - A list of floats representing the YoY growth percentage for each year compared to the previous year.
    """
    # Ensure there is sufficient data for year-on-year calculation
    if len(sales_data) < 2:
        return ["Not enough data to calculate year-on-year sales growth."]
    
    yoy_growth = []
    
    # Sort the sales data by year to ensure proper calculation
    sales_data_sorted = sorted(zip(year_data, sales_data), key=lambda x: x[0])

    # Extract sorted years and sales
    sorted_years, sorted_sales = zip(*sales_data_sorted)

    for i in range(1, len(sorted_years)):
        current_year = sorted_years[i]
        previous_year = sorted_years[i - 1]

        current_year_sales = sorted_sales[i]
        previous_year_sales = sorted_sales[i - 1]

        if previous_year_sales is not None and current_year_sales is not None:
            # Calculate YoY growth
            growth = ((current_year_sales - previous_year_sales) / previous_year_sales) * 100
            yoy_growth.append(growth)
        else:
            yoy_growth.append(None)  # If data is missing
    
    return yoy_growth


In [93]:
tools = [Extract_Tool,Growth_Tool]

In [94]:
llm = ChatOpenAI(model="gpt-4o", openai_api_key=key_string)

In [95]:
prompt = hub.pull("hwchase17/openai-tools-agent")

/Users/gunik/anaconda3/lib/python3.11/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [96]:
agent = create_tool_calling_agent(
    llm=llm,  # Language model to use
    tools=tools,  # List of tools available to the agent
    prompt=prompt,  # Prompt template to guide the agent's responses
)

In [97]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,  # The agent to execute
    tools=tools,  # List of tools available to the agent
    verbose=True,  # Enable verbose logging
    handle_parsing_errors=True,  # Handle parsing errors gracefully
)

In [98]:
response = agent_executor.invoke({"input": " extract net sales of Apple 2019 to 2023 "})



> Entering new AgentExecutor chain...

Invoking: `Extract_Tool` with `{'a': 'Apple', 'b': 'Net Sales', 'c': '2019', 'd': '2023'}`


([260174, 274515, 365817, 394328, 383285], [2019, 2020, 2021, 2022, 2023])The net sales of Apple from 2019 to 2023 are as follows:

- 2019: $260,174 million
- 2020: $274,515 million
- 2021: $365,817 million
- 2022: $394,328 million
- 2023: $383,285 million

> Finished chain.


In [106]:
response = agent_executor.invoke({"input": "give me insights on sales growth from 2021 to 2024 for apple"})



> Entering new AgentExecutor chain...

Invoking: `Extract_Tool` with `{'a': 'Apple', 'b': 'Net Sales', 'c': '2021', 'd': '2024'}`


([365817, 394328, 383285, 391035], [2021, 2022, 2023, 2024])
Invoking: `Growth_Tool` with `{'sales_data': [365817, 394328, 383285, 391035], 'year_data': [2021, 2022, 2023, 2024]}`


[7.7937876041846055, -2.8004605303199366, 2.021994077514121]Here are the insights on Apple's sales growth from 2021 to 2024:

- **2021 to 2022**: Apple experienced a growth in net sales of approximately **7.79%**.
- **2022 to 2023**: There was a decline in net sales by about **2.80%**.
- **2023 to 2024**: Sales increased slightly with a growth of around **2.02%**.

Overall, Apple's net sales showed fluctuations over these years, with a notable growth in 2022 followed by a slight decline in 2023 and a small recovery in 2024.

> Finished chain.


In [107]:
print(response)

{'input': 'give me insights on sales growth from 2021 to 2024 for apple', 'output': "Here are the insights on Apple's sales growth from 2021 to 2024:\n\n- **2021 to 2022**: Apple experienced a growth in net sales of approximately **7.79%**.\n- **2022 to 2023**: There was a decline in net sales by about **2.80%**.\n- **2023 to 2024**: Sales increased slightly with a growth of around **2.02%**.\n\nOverall, Apple's net sales showed fluctuations over these years, with a notable growth in 2022 followed by a slight decline in 2023 and a small recovery in 2024."}
